In [1]:
# TODO Turn this into a .py file so that the client can run it.

In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

2023-12-12 10:29:45.376348: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 10:29:45.376392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 10:29:45.378390: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 10:29:45.552460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Be a good neighbor. Let other people use the GPU too. 
# print(tf.config.list_physical_devices('GPU'))
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
# Load up the training data
data_file_location = 'training_data/'
data_file_name = 'data03_train.npy'
label_file_name = 'labels03_train.npy'

checkpoint_filepath = 'temp/checkpoint/convnet03_2500.h5'
log_filepath = 'temp/tb_logs'

data = np.load(data_file_location+data_file_name)
labels = np.load(data_file_location+label_file_name)

In [5]:
x = data
y = labels[:,0]

In [6]:
def build_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size, activation='softmax'))
    return model
    

In [7]:
def build_simple_model(input_shape, output_size):
    model = Sequential()
    model.add(Conv1D(1, kernel_size=3, padding='same', input_shape=input_shape, activation='relu'))
    model.add(Flatten())
    model.add(Dense(output_size, activation='softmax'))
    return model

In [8]:
model = build_model(input_shape=(1024,8),output_size=25)
model.summary()

2023-12-12 10:29:50.857413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 10:29:50.897043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 10:29:50.897085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 10:29:50.900524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-12 10:29:50.900565: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1024, 10)          410       
                                                                 
 max_pooling1d (MaxPooling1  (None, 512, 10)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 512, 20)           1020      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 256, 20)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 256, 30)           3030      
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 128, 30)           0

In [9]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [10]:
# Setup the callbacks
checkpt = ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=0)
tb = TensorBoard(log_dir=log_filepath)
e_stop = EarlyStopping(patience=5)

In [11]:
# Train the model
hist = model.fit(x, y, epochs=100, validation_split=0.2, verbose=2, batch_size=100,
                 callbacks=[checkpt, tb, e_stop])

Epoch 1/100


2023-12-12 10:29:58.363499: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-12 10:30:01.612890: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff35d1870c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-12 10:30:01.612939: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro T1000, Compute Capability 7.5
2023-12-12 10:30:01.626324: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-12 10:30:01.762261: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/home/thomas/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is conside

2/2 - 12s - loss: 3.2240 - accuracy: 0.0550 - val_loss: 3.2111 - val_accuracy: 0.0400 - 12s/epoch - 6s/step
Epoch 2/100
2/2 - 0s - loss: 3.2086 - accuracy: 0.0550 - val_loss: 3.2268 - val_accuracy: 0.0600 - 100ms/epoch - 50ms/step
Epoch 3/100
2/2 - 0s - loss: 3.1986 - accuracy: 0.0600 - val_loss: 3.2433 - val_accuracy: 0.0600 - 102ms/epoch - 51ms/step
Epoch 4/100
2/2 - 0s - loss: 3.1861 - accuracy: 0.0650 - val_loss: 3.2701 - val_accuracy: 0.0600 - 86ms/epoch - 43ms/step
Epoch 5/100
2/2 - 0s - loss: 3.1847 - accuracy: 0.0650 - val_loss: 3.2830 - val_accuracy: 0.0600 - 93ms/epoch - 47ms/step
Epoch 6/100
2/2 - 0s - loss: 3.1772 - accuracy: 0.0700 - val_loss: 3.2617 - val_accuracy: 0.0600 - 110ms/epoch - 55ms/step
